In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import Hour, Minute
from pandas.tseries.offsets import Day, MonthEnd
from pandas.tseries.offsets import Hour
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from keras.preprocessing.sequence import TimeseriesGenerator
import xgboost as xgb

np.random.seed(42)


In [2]:

global busses
busses =        ['310_PV_1', '310_PV_2', '320_PV_1', '314_PV_1', '314_PV_2', '313_PV_1', '314_PV_3', '314_PV_4', '313_PV_2', '324_PV_1',    
                 '312_PV_1', '324_PV_2', '324_PV_3', '113_PV_1', '319_PV_1', '215_PV_1', '102_PV_1', '101_PV_1', '102_PV_2',
                 '104_PV_1', '101_PV_2', '101_PV_3', '101_PV_4', '103_PV_1', '119_PV_1']
df = pd.read_csv('PV_deseason_15.csv', parse_dates=[0], index_col=0)


In [3]:
def lags(df, n, bus):
    
    df_new = df
    
    for i in range(n):
        name = "lag "+ str(i+1)
        df_new[name] = df[bus].shift(i+1)
    
    return df_new
    

In [4]:
def WindowSplit(n, df):
    ##last day is considered counted by n the number of days (n-1 train, 1 test)
    indices = list(range(len(df.index)))
    result = []
    
    freq = n*96
    
    total_days = len(df.index)//96
    windows = total_days - n + 1
    
    
    start = 0
    for i in range(windows):
        folds = []
        for j in range(5):
            end = start+(n-2)*96 + 91 + 20*j +1
            train = indices[start:end]
            test = [indices[end]]
            folds.append((train,test))
            
        result.append(folds)
        start += 96
        
    return result, windows

In [5]:



def model1(ws,windows,scaler,df):
    mse1 = []

    for window in ws:
        
        for train_indx, val_indx in window:
            train = df.iloc[train_indx]
            test = df.iloc[val_indx]
        
            #lag = pd.concat(train, test)
        
            lag = lags(df, 3, '310_PV_1')
        
            lagged_test = lag.iloc[val_indx]
            lagged_train = lag.iloc[train_indx]
        
        
        
        
            X_train = lagged_train.drop(busses, axis = 1)
            y_train = lagged_train['310_PV_1']
        
            X_test = lagged_test.drop(busses, axis = 1)
            y_test = lagged_test['310_PV_1']
        
    
        
            mod = xgb.XGBRegressor()
        
       
        
            mod.fit(X_train, y_train)#keep low for now, edit later eval_set = [(X_train, y_train),(X_test, y_test)]
        
            final_batch_X = X_test.head(1)
            final_batch_y = y_test.head(1)
        
    
            pred = mod.predict(final_batch_X)
    
        
            error = mse([test['310_PV_1'][0]],pred)
            mse1.append(error)

    #print(mse)
    return mse1

    

In [6]:
def modelNaive(tss,df):

    mse3 = []

    for train_indx, val_indx in tss.split(df):
        train = df.iloc[train_indx]
        test = df.iloc[val_indx]
    
        pred = train['310_PV_1'][-1]
    
    
        error = 0.5*(test['310_PV_1'][0] - pred)**2
        mse3.append(error)
        
    print(mse3)
    return mse3

In [7]:
def runModels(df):
    ws, windows =  WindowSplit(5,df)#window split
    scaler = MinMaxScaler()
    mse1 = model1(ws,windows,scaler,df)
    
    return mse1

    

In [8]:

df1 = df[df.index.month.isin([1,2,3])]
df2 = df[df.index.month.isin([4,5,6])]
df3 = df[df.index.month.isin([7,8,9])]
df4 = df[df.index.month.isin([10,11,12])]

In [9]:
mse1 = runModels(df1)

C:\Users\bvits\AppData\Local\Temp\ipykernel_46600\788086416.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[name] = df[bus].shift(i+1)
C:\Users\bvits\AppData\Local\Temp\ipykernel_46600\788086416.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[name] = df[bus].shift(i+1)
C:\Users\bvits\AppData\Local\Temp\ipykernel_46600\788086416.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

9.892466016014579e-09


AttributeError: 'function' object has no attribute 'append'

In [ ]:
mse2 = runModels(df2)

In [ ]:
mse3 = runModels(df3)

In [ ]:
mse4 = runModels(df4)

In [ ]:
print('season 1')

print('model 1:')
print(mse1)



In [ ]:
print('season 1')

print('model 1:')
print(mse2)


In [ ]:
print('season 1')

print('model 1:')
print(mse3)



In [ ]:
print('season 1')

print('model 1:')
print(mse4)



In [ ]:
d = {'Season 1':mse1,'Season 2':mse2,'Season 3':mse3,'Season 4':mse4}
df_errors_GBDT = pd.DataFrame({k:pd.Series(v) for k,v in d.items()})

df_errors_GBDT.to_csv('GBDT_errors.csv')
